In [ ]:
import os
from base64 import b64encode
from abc_project_vars import DIR_AGENT_PROMPTS
from abc_project_vars import PROMPT_FILES
from mistralai import Mistral
from utilities_io import load_file_as_string

In [2]:
def encode_image(image_path):
    """Encode the image to base64."""
    try:
        with open(image_path, "rb") as image_file:
            return b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"Error: The file {image_path} was not found.")
        return None
    except Exception as e:  # Added general exception handling
        print(f"Error: {e}")
        return None

In [3]:
# Setup the API key, client and model
api_key = os.environ["MISTRAL_API_KEY"]
client  = Mistral(api_key=api_key)
model   = "pixtral-12b-2409"
# Read agent prompt
prompt_path = os.path.join( DIR_AGENT_PROMPTS, AGENT_PROMPTS[0])
prompt = load_file_as_string(prompt_path)

In [ ]:
# Path to your image
IMAGE_DIR = "/home/luis/DJI_AGRAS_LATINO/rc/t40_t50/flight/"
# Spray images
image_name = "2023-05-31_09-11-36_i0.jpg"
# image_name = "2023-09-07_20-38-33_i0.jpg"
# image_name = "2024-01-11_09-36-34_i0.jpg"
# image_name = "2023-07-10_14-12-29_i0.jpg"
# image_name = "2023-07-23_18-29-39_i0.jpg"
# image_name = "2023-07-25_13-45-14_i0.jpg"
# Prop images
# image_name = "2023-06-13_16-33-59_i0.jpg"
# image_name = "2023-09-05_07-13-10_i0.jpg"
# Flight images
# image_name = "2023-07-22_10-22-35_i0.jpg"
# image_name = "2023-07-25_11-08-08_i0.jpg"
# image_name = "2023-08-23_11-05-18_i0.jpg"
# image_name = "2023-08-26_11-53-42_i0.jpg"

In [33]:
# Encode image as base64
base64_image = encode_image(IMAGE_DIR + image_name)
# Define the messages for the chat
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            },
            {
                "type": "image_url",
                "image_url": f"data:image/jpeg;base64,{base64_image}" 
            }
        ]
    }
]
# Get the chat response
try:
    chat_response = client.chat.complete(
        model=model,
        messages=messages # type: ignore
    )
    if chat_response and chat_response.choices:
        # Print the content of the response
        extracted_errors = str(chat_response.choices[0].message.content)
        print(extracted_errors)
    else:
        print("No response received from the API")   
except Exception as e:
    print(f"Error calling API: {e}")

```json
{
    "metadata": {
        "language": "Spanish",
        "num_msg": 4
    },
    "data": [
        "Rociado/Área atrapado en el tubo",
        "Error del sistema de electrónica aérea",
        "Error del sistema de electrónica aérea",
        "Error del sistema de electrónica aérea",
        "Error de comunicación interna de radar II"
    ]
}
```


In [34]:
from utilities_io import load_json_string
error_obj = load_json_string(extracted_errors)
language = error_obj['metadata']['language']
num_msg  = error_obj['metadata']['num_msg']
print(f"Language: {language}")
print(f"Number of messages: {num_msg}")

if error_obj['data'] :
    for msg in error_obj['data']:
        print(msg)

Language: Spanish
Number of messages: 4
Rociado/Área atrapado en el tubo
Error del sistema de electrónica aérea
Error del sistema de electrónica aérea
Error del sistema de electrónica aérea
Error de comunicación interna de radar II


In [35]:
from utilities_io import save_json_file
fname = 'stage_1_results/' + image_name.replace(".jpg", ".json")
save_json_file( fname, error_obj)